<h1>SUBSET SELECTION</h1>

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy
import sklearn
import seaborn as sns
import xlrd
import time
import statsmodels.api as sm

In [4]:
data=pd.read_excel('Data/Mini Project EFSA.xlsx')
data.rename(columns={'sex \n(0=M, 1=F)':'sex'}, inplace=True)
data

,response,number of animals,SD,dose,sex,endpoint
0,570.4,9,75.6,0,0,1
1,548.5,10,44.3,30,0,1
2,533.0,10,72.3,100,0,1
3,488.1,9,42.5,150,0,1
4,329.8,10,46.8,0,1,1
5,319.7,10,24.3,30,1,1
6,293.3,10,20.1,100,1,1
7,312.4,10,43.1,150,1,1
8,45.2,9,1.5,0,0,2
9,44.6,10,1.7,30,0,2


In [35]:
def processSubset(feature_set,weights):
    # Fit model on feature_set and calculate RSS
    #model = sm.OLS(y,X[list(feature_set)])
    model = sm.WLS(y,X[list(feature_set)],weights=weights)
    regr = model.fit()
    RSS = ((regr.predict(X[list(feature_set)]) - y) ** 2).sum()
    return {"model":regr, "RSS":RSS}

In [34]:
def forward(predictors,weights):

    # Pull out predictors we still need to process
    remaining_predictors = [p for p in X.columns if p not in predictors]
    
    tic = time.time()
    
    results = []
    
    for p in remaining_predictors:
        results.append(processSubset(predictors+[p],weights))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.loc[models['RSS'].argmin()]
    
    toc = time.time()
    print("Processed ", models.shape[0], "models on", len(predictors)+1, "predictors in", (toc-tic), "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model

<h2>I dati sono le colonne originali</h2>

In [36]:
#Prepare the datas
y = data.response
weights = data.SD
X = data.drop(columns=["response","SD"])

In [37]:
models_fwd = pd.DataFrame(columns=["RSS", "model"])

tic = time.time()
predictors = []

for i in range(1,len(X.columns)+1):    
    models_fwd.loc[i] = forward(predictors,weights)
    predictors = models_fwd.loc[i]["model"].model.exog_names

toc = time.time()
print("Total elapsed time:", (toc-tic), "seconds.")

Processed  4 models on 1 predictors in 0.021669864654541016 seconds.
Processed  3 models on 2 predictors in 0.013873100280761719 seconds.
Processed  2 models on 3 predictors in 0.005861997604370117 seconds.
Processed  1 models on 4 predictors in 0.0 seconds.
Total elapsed time: 0.06980133056640625 seconds.


In [38]:
display(models_fwd)

,RSS,model
1,1.747568e+06,<statsmodels.regression.linear_model.Regressio...
2,2.196260e+06,<statsmodels.regression.linear_model.Regressio...
3,2.474017e+06,<statsmodels.regression.linear_model.Regressio...
4,4.712230e+05,<statsmodels.regression.linear_model.Regressio...


In [39]:
models_fwd.RSS

1    1.747568e+06
2    2.196260e+06
3    2.474017e+06
4    4.712230e+05
Name: RSS, dtype: float64